# Evaluación del Sistema de Streaming - World Happiness Report

Este notebook evalúa el desempeño del sistema completo:
- **Kafka Producer/Consumer** en tiempo real
- **Modelo de Machine Learning** (Regresión Lineal)
- **Almacenamiento en MySQL**

---

In [23]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

---
## 1. Cargar Datos desde MySQL

Conectamos a la base de datos MySQL donde se almacenaron las predicciones del streaming.

In [24]:
# Configuración de conexión MySQL
mysql_config = {
    'host': 'localhost',
    'port': 3306,
    'database': 'happiness_db',
    'user': 'root',
    'password': 'root'
}

# Conectar a MySQL
try:
    conn = mysql.connector.connect(**mysql_config)
    print(f"Conectado a MySQL: {mysql_config['database']}")
    
    # Cargar datos de predicciones
    query = "SELECT * FROM predictions"
    df_predictions = pd.read_sql(query, conn)
    
    print(f"\nTotal de predicciones almacenadas: {len(df_predictions)}")
    print(f"\nPrimeras filas:")
    display(df_predictions.head())
    
    conn.close()
    
except Exception as e:
    print(f"Error al conectar a MySQL: {e}")
    print("Asegúrate de que MySQL esté corriendo y las credenciales sean correctas")

Conectado a MySQL: happiness_db

Total de predicciones almacenadas: 205

Primeras filas:


C:\Users\juana\AppData\Local\Temp\ipykernel_12348\3027414067.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_predictions = pd.read_sql(query, conn)


,id,record_id,timestamp,country,year,gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_life_choices,generosity,perceptions_of_corruption,actual_score,predicted_score,prediction_error,processing_time_ms,created_at
0,1,0,2025-11-03 03:02:00,Switzerland,2015,1.39651,1.34951,0.94143,0.66557,0.29678,0.41978,7.587,7.16444,0.422556,24.311500,2025-11-03 03:02:00
1,2,1,2025-11-03 03:02:00,Iceland,2015,1.30232,1.40223,0.94784,0.62877,0.43630,0.14145,7.561,6.85395,0.707047,0.354767,2025-11-03 03:02:00
2,3,2,2025-11-03 03:02:00,Denmark,2015,1.32548,1.36058,0.87464,0.64938,0.34139,0.48357,7.527,7.08540,0.441603,0.339270,2025-11-03 03:02:00
3,4,3,2025-11-03 03:02:01,Norway,2015,1.45900,1.33095,0.88521,0.66973,0.34699,0.36503,7.522,7.14171,0.380287,0.349045,2025-11-03 03:02:01
4,5,4,2025-11-03 03:02:01,Canada,2015,1.32629,1.32261,0.90563,0.63297,0.45811,0.32957,7.427,6.98537,0.441627,0.328779,2025-11-03 03:02:01


In [25]:
# Información del dataset
print(f"\nDimensiones: {df_predictions.shape}")
print(f"Columnas: {df_predictions.columns.tolist()}")
print(f"\nTipos de datos:")
print(df_predictions.dtypes)
print(f"\nValores nulos:")
print(df_predictions.isnull().sum())


Dimensiones: (205, 16)
Columnas: ['id', 'record_id', 'timestamp', 'country', 'year', 'gdp_per_capita', 'social_support', 'healthy_life_expectancy', 'freedom_to_make_life_choices', 'generosity', 'perceptions_of_corruption', 'actual_score', 'predicted_score', 'prediction_error', 'processing_time_ms', 'created_at']

Tipos de datos:
id                                       int64
record_id                                int64
timestamp                       datetime64[ns]
country                                 object
year                                     int64
gdp_per_capita                         float64
social_support                         float64
healthy_life_expectancy                float64
freedom_to_make_life_choices           float64
generosity                             float64
perceptions_of_corruption              float64
actual_score                           float64
predicted_score                        float64
prediction_error                       float64
processing

---
## 2. Métricas de Evaluación del Modelo

Calculamos las métricas de desempeño del modelo sobre las predicciones en streaming.

In [26]:
# Calcular métricas
y_true = df_predictions['actual_score']
y_pred = df_predictions['predicted_score']

# Métricas
r2 = r2_score(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Error promedio
error_promedio = df_predictions['prediction_error'].mean()

print("="*80)
print("MÉTRICAS DE DESEMPEÑO - STREAMING CON KAFKA")
print("="*80)
print(f"\nR² (Coefficient of Determination):    {r2:.4f}")
print(f"MAE (Mean Absolute Error):            {mae:.4f}")
print(f"RMSE (Root Mean Squared Error):       {rmse:.4f}")
print(f"MAPE (Mean Absolute Percentage Error): {mape:.2f}%")
print(f"\nError Promedio de Predicción:         {error_promedio:.4f}")
print(f"\n{'='*80}")

# Interpretación
print("\nINTERPRETACIÓN:")
print(f"   - El modelo explica el {r2*100:.2f}% de la variabilidad del Happiness Score")
print(f"   - Error absoluto promedio: {mae:.4f} puntos (en escala 0-10)")
print(f"   - Error porcentual relativo: {mape:.2f}%")

MÉTRICAS DE DESEMPEÑO - STREAMING CON KAFKA

R² (Coefficient of Determination):    0.5580
MAE (Mean Absolute Error):            0.4349
RMSE (Root Mean Squared Error):       0.5432
MAPE (Mean Absolute Percentage Error): 7.20%

Error Promedio de Predicción:         0.4349


INTERPRETACIÓN:
   - El modelo explica el 55.80% de la variabilidad del Happiness Score
   - Error absoluto promedio: 0.4349 puntos (en escala 0-10)
   - Error porcentual relativo: 7.20%


---
## 3. Exportar Datos y Métricas

Los datos se exportan para ser visualizados en el dashboard de KPIs.

In [27]:
# Crear carpeta data si no existe
os.makedirs('../data', exist_ok=True)

# Exportar predicciones completas
df_predictions.to_csv('../data/predictions_streaming.csv', index=False)
print(f"Tabla exportada: predictions_streaming.csv")

# Exportar métricas resumidas
metricas_df = pd.DataFrame({
    'Métrica': ['R²', 'MAE', 'RMSE', 'MAPE (%)', 'Error Promedio', 'Tiempo Procesamiento (ms)'],
    'Valor': [r2, mae, rmse, mape, error_promedio, df_predictions['processing_time_ms'].mean()]
})
metricas_df.to_csv('../data/metricas_resumen.csv', index=False)
print(f"Métricas exportadas: metricas_resumen.csv")

Tabla exportada: predictions_streaming.csv
Métricas exportadas: metricas_resumen.csv

RESUMEN DE MÉTRICAS


,Métrica,Valor
0,R²,0.557993
1,MAE,0.434905
2,RMSE,0.543246
3,MAPE (%),7.203210
4,Error Promedio,0.434906
5,Tiempo Procesamiento (ms),0.630209



NOTA: Para visualizar los KPIs, ejecutar:
  cd dashboard
  python dashboard_kpis.py
